# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-1181ce72bf-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



In [2]:
# start your workflow here

## Task 1

In [3]:
queryString = """
SELECT *
WHERE { 

wd:Q25089 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('obj', 'http://data.bibliotheken.nl/id/thes/p06917654X')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('obj', 'https://livedata.bibsys.no/authority/90085344')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1150'), ('obj', 'http://rvk.uni-regensburg.de/nt/HU+3027')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741'), ('obj', 'http://data.beeldengeluid.nl/gtaa/76199')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/59077912')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('obj', 'http://id.worldcat.org/fast/1897483')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('obj', 'https://d-nb.info/gnd/118502077')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.go

334

I notice that P31, P166, P800 and P106 are recurrent properties. 

In [ ]:
queryString = """
SELECT ?p ?name 
WHERE { 

wd:Q25089 ?p wd:Q5 .
?p <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)



Results


In [5]:
queryString = """
SELECT ?name 
WHERE { 

wd:Q25089 ?p wd:Q5 .
wdt:P166 <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'award received')]


1

In [6]:
queryString = """
SELECT ?name 
WHERE { 

wd:Q25089 ?p wd:Q5 .
wdt:P800 <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'notable work')]


1

In [7]:
queryString = """
SELECT ?name 
WHERE { 

wd:Q25089 ?p wd:Q5 .
wdt:P106 <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'occupation')]


1

Here I look at all the occupations of Woody Allen thanks to the property P106 "occupation"

In [8]:
queryString = """
SELECT ?obj ?name 
WHERE { 

wd:Q25089 wdt:P106 ?obj .
?obj <http://schema.org/name> ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q15981151'), ('name', 'jazz musician')]
[('obj', 'http://www.wikidata.org/entity/Q18844224'), ('name', 'science fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist')]
[('obj', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('obj', 'http://www.wikidata.org/entity/Q245068'), ('name', 'comedian')]
[('obj', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('obj', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('obj', 'http://www.wikidata.org/entity/Q639669'), ('name', 'm

15

Some film directors

In [9]:
queryString = """
SELECT ?person ?personName ?p ?pName 
WHERE { 

?person ?p wd:Q2526255.

?person <http://schema.org/name> ?personName .
?p <http://schema.org/name> ?pName .
} 
limit 30
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q15278146'), ('personName', 'Template:Infobox film director'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('person', 'http://www.wikidata.org/entity/Q18416268'), ('personName', 'Mario Ribero Ferreira'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q16397'), ('personName', 'Bruce Lee'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q361670'), ('personName', 'John Sturges'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q6122250'), ('personName', 'Jagathy Sreekumar'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('person', 'http://www.wikidata.org/entity/Q7238182'), ('personName', 'Prakash Kovelamudi'), ('p', 'http://www.wikidata.or

30

I use P800 "notable work" to discover all works, not necessarily movies, of Woody Allen

In [10]:
queryString = """
SELECT ?obj ?name 
WHERE { 

wd:Q25089 wdt:P800 ?obj .

?obj <http://schema.org/name> ?name .

} 
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
[('obj', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway')]
[('obj', 'http://www.wikidata.org/entity/Q1354109'), ('name', 'Mighty Aphrodite')]
[('obj', 'http://www.wikidata.org/entity/Q729026'), ('name', 'Broadway Danny Rose')]
[('obj', 'http://www.wikidata.org/entity/Q733677'), ('name', 'Match Point')]
[('obj', 'http://www.wikidata.org/entity/Q740143'), ('name', 'Husbands and Wives')]
[('obj', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas')]
[('obj', 'http://www.wikidata.org/entity/Q1321004'), ('name', 'You Will Meet a Tall Dark Stranger')]
[('obj', 'http://www.wikidata.org/entity/Q14511869'), ('name', 'Magic in the Moonlight')]
[('obj', 'http://www.wikidata.org/entity/Q614009'), ('name', 'Anything Else')]
[('obj', 'http://www.wikidata.org/entity/Q187423'), ('name', 

49

Test 1 - No new information

In [11]:
queryString = """
SELECT ?pName ?objName ?name 
WHERE { 

wd:Q25089 ?p ?obj .
?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

FILTER (?p = wdt:P800 || ?p = wdt:P106)

} 
"""

print("Results")
run_query(queryString)

Results
[('pName', 'occupation'), ('objName', 'film actor')]
[('pName', 'occupation'), ('objName', 'jazz musician')]
[('pName', 'occupation'), ('objName', 'science fiction writer')]
[('pName', 'occupation'), ('objName', 'journalist')]
[('pName', 'occupation'), ('objName', 'playwright')]
[('pName', 'occupation'), ('objName', 'comedian')]
[('pName', 'occupation'), ('objName', 'film director')]
[('pName', 'occupation'), ('objName', 'screenwriter')]
[('pName', 'occupation'), ('objName', 'film producer')]
[('pName', 'occupation'), ('objName', 'writer')]
[('pName', 'occupation'), ('objName', 'composer')]
[('pName', 'occupation'), ('objName', 'author')]
[('pName', 'occupation'), ('objName', 'musician')]
[('pName', 'occupation'), ('objName', 'character actor')]
[('pName', 'occupation'), ('objName', 'clarinetist')]
[('pName', 'notable work'), ('objName', 'Midnight in Paris')]
[('pName', 'notable work'), ('objName', "What's Up, Tiger Lily?")]
[('pName', 'notable work'), ('objName', 'Bullets Over

64

Better undertanding of all the name of the properties and objects of Woody Allen. 

In [12]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q25089 ?p ?obj .
?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('objName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10800557'), ('objName', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q15981151'), ('objName', 'jazz musician')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q18844224'), ('objName', 'science fiction writer')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q1930187'), ('objName', 'journalist')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q214917'), ('objName', 'playwright')]
[('p', 'http://www.wikidata.org/prop/dir

134

Find properties and objects of W. Allen's works

In [13]:
queryString = """
SELECT DISTINCT ?work ?workName ?p ?pName ?obj ?objName
WHERE { 

wd:Q25089 wdt:P800 ?work .
?work ?p ?obj .

?work <http://schema.org/name> ?workName .
?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

} 
"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q1052826'), ('workName', 'Celebrity'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q22963890'), ('objName', 'Angel Caban')]
[('work', 'http://www.wikidata.org/entity/Q1052826'), ('workName', 'Celebrity'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q19668406'), ('objName', 'Yolonda Ross')]
[('work', 'http://www.wikidata.org/entity/Q614009'), ('workName', 'Anything Else'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q184103'), ('objName', 'Christina Ricci')]
[('work', 'http://www.wikidata.org/entity/Q806092'), ('workName', 'Bananas'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q448005'), ('objName', 'David Ortiz')]
[('work', 'http://www.wikidata.org/

2419

Explore the properties of Irrational Man "Q17417520"

In [14]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q17417520 ?p ?obj .


?p <http://schema.org/name> ?pName .
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
LIMIT 15
"""
#?obj <http://schema.org/name> ?objName .
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5201'), ('pName', 'IMDA rating'), ('obj', 'http://www.wikidata.org/entity/Q52382306'), ('objName', 'NC16')]
[('p', 'http://www.wikidata.org/prop/direct/P915'), ('pName', 'filming location'), ('obj', 'http://www.wikidata.org/entity/Q54264'), ('objName', 'Newport')]
[('p', 'http://www.wikidata.org/prop/direct/P2629'), ('pName', 'BBFC rating'), ('obj', 'http://www.wikidata.org/entity/Q23301856'), ('objName', '12 certificate')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q3161523'), ('objName', 'Jamie Blackley')]
[('p', 'http://www.wikidata.org/prop/direct/P6658'), ('pName', 'RCQ classification'), ('obj', 'http://www.wikidata.org/entity/Q62365366'), ('objName', 'G')]
[('p', 'http://www.wikidata.org/prop/direct/P2684'), ('pName', 'Kijkwijzer rating'), ('obj', 'http://www.wikidata.org/entity/Q23649981'), ('objName', '6')]
[('p', 'http://www.wikidata.org/prop/direct/P

15

**1 - BGP films**
From the previous query I discover the film class "Q11424". Explore its properties

## Task 2

In [15]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q11424 ?p ?obj .

?p <http://schema.org/name> ?pName.
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pName', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pName', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pName', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pName', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('pName', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('pName', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main templat

75

Properties of film class with obj name

In [16]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q11424 ?p ?obj .
?obj <http://schema.org/name> ?objName .
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('obj', 'http://www.wikidata.org/entity/Q1660187'), ('objName', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('obj', 'http://www.wikidata.org/entity/P345'), ('objName', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('obj', 'http://www.wikidata.org/entity/P1040'), ('objName', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('obj', 'http://www.wikidata.org/entity/P344'), ('objName', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('obj', 'http://www.wikidata.org/entity/Q17738'), ('objName', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('obj', 'http://www.wikidata.org/entity/Q55321'), ('objName', 'Category:Films')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q20937557'), ('objName', 'series')]
[('p', 'http://www.wikidata.org/prop/direct/P19

65

Properties and objects found in the above query with their names.

In [17]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q11424 ?p ?obj .
?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName
} 

"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q3296835'), ('objName', 'Portal:Film')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q2657718'), ('objName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q6171351'), ('objName', 'Template:Infobox film')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q105759907'), ('objName', 'film reel')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj

65

Look at the cinema class "Q16144339"

In [18]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q16144339 ?p ?obj .

OPTIONAL{?p <http://schema.org/name> ?pName}
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1190'), ('pName', 'Universal Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('pName', 'partially coincident with')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2888'), ('pName', 'exact match')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as')]
[('p', 'http://www.wikidata.org

22

Search for cinema its class

In [19]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q11424 wdt:P279* ?obj .
?obj ?p ?obj2 .
?p <http://schema.org/name> ?pName .
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pName', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pName', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pName', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P111'), ('pName', 'measured physical quantity')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pName', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('pName', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('pName', 'OpenStreetMap tag or key')]
[('p'

141

I am not satisfied with previous research. I still have to find the BUDGET property or somenthing similar. I explore the properties of the instances of the film class.

In [20]:
queryString = """
SELECT DISTINCT ?p2 ?p2Name
WHERE { 

wd:Q11424 ?p ?obj .
?obj ?p2 ?obj2 .
?p2 <http://schema.org/name> ?p2Name.
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p2', 'http://www.wikidata.org/prop/direct/P2876'), ('p2Name', 'type of unit for this property')]
[('p2', 'http://www.wikidata.org/prop/direct/P3851'), ('p2Name', 'Cinenacional.com movie ID')]
[('p2', 'http://www.wikidata.org/prop/direct/P8920'), ('p2Name', 'Movist film ID')]
[('p2', 'http://www.wikidata.org/prop/direct/P1423'), ('p2Name', 'template has topic')]
[('p2', 'http://www.wikidata.org/prop/direct/P5201'), ('p2Name', 'IMDA rating')]
[('p2', 'http://www.wikidata.org/prop/direct/P8918'), ('p2Name', 'Max Movie film ID')]
[('p2', 'http://www.wikidata.org/prop/direct/P4437'), ('p2Name', 'FPB rating')]
[('p2', 'http://www.wikidata.org/prop/direct/P9236'), ('p2Name', 'RDA value vocabularies ID')]
[('p2', 'http://www.wikidata.org/prop/direct/P1628'), ('p2Name', 'equivalent property')]
[('p2', 'http://www.wikidata.org/prop/direct/P8889'), ('p2Name', 'MTRCB rating')]
[('p2', 'http://www.wikidata.org/prop/direct/P2236'), ('p2Name', 'external subproperty')]
[('p2', 'http://www.w

300

Look for "budget"

In [21]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName  
WHERE { 

wd:Q11424 ?p ?obj .

OPTIONAL{?obj <http://schema.org/name> ?objName .
            ?p <http://schema.org/name> ?pName}
            
FILTER(REGEX(?pName, ".*get"))
} 

"""

print("Results")
run_query(queryString)

Results
Empty


0

bgp director as property

In [22]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:P57 ?p ?obj .

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q18618644'), ('objName', 'Wikidata property for items about works')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('obj', 'http://www.wikidata.org/entity/P50'), ('objName', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('obj', 'http://www.wikidata.org/entity/P170'), ('objName', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P1647'), ('pName', 'subproperty of'), ('obj', 'http://www.wikidata.org/entity/P50'), ('objName', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P1647'), ('pName', 'subproperty of'), ('obj', 'http://www.wikidata.org/entity/P3342'), ('objName', 'significant person')]
[('p', 'http://www.

18

**2 - BGP film directors**
query 13 is wrong. I need to focus on director as entity. From Woody Allen occupations I discover Q2526255 as film director.

## Task 3

In [23]:
queryString = """
SELECT ?p ?pName ?obj ?objName 
WHERE { 

wd:Q2526255 ?p ?obj.

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .
} 
limit 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q15278146'), ('objName', 'Template:Infobox film director')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q222344'), ('objName', 'cinematographer')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1969'), ('objName', 'MovieMeter director ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3406858'), ('objName', 'film professional')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3391

12

From the previous query, Q4220920 is the filmmaking occupation class. I may have an answer for the BGP of the worker in movies.

In [24]:
queryString = """
SELECT DISTINCT ?entity ?entityName ?p
WHERE { 

?entity ?p wd:Q4220920 .
?entity <http://schema.org/name> ?entityName .
} 
ORDER BY ASC (?entityName)
"""

print("Results")
run_query(queryString)

Results
[('entity', 'http://www.wikidata.org/entity/Q108137233'), ('entityName', 'Calibration'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('entity', 'http://www.wikidata.org/entity/Q2933935'), ('entityName', 'Camera resectioning'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('entity', 'http://www.wikidata.org/entity/Q9727432'), ('entityName', 'Category:Filmmaking occupations'), ('p', 'http://www.wikidata.org/prop/direct/P301')]
[('entity', 'http://www.wikidata.org/entity/Q10495176'), ('entityName', 'Filmare'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('entity', 'http://www.wikidata.org/entity/Q1639269'), ('entityName', 'Focus puller'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('entity', 'http://www.wikidata.org/entity/Q60750837'), ('entityName', 'Intimacy coordinator'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('entity', 'http://www.wikidata.org/entity/Q108128082'), ('entityName', 'Sound design'), ('p', 'http://www.wikidata.org/prop/direc

82

film professional Q3406858

In [25]:
queryString = """
SELECT DISTINCT ?entity ?entityName 
WHERE { 

?entity ?p wd:Q3406858 .
?entity <http://schema.org/name> ?entityName .
} 

"""

print("Results")
run_query(queryString)

Results
[('entity', 'http://www.wikidata.org/entity/Q4220920'), ('entityName', 'filmmaking occupation')]
[('entity', 'http://www.wikidata.org/entity/Q58623063'), ('entityName', 'film distributor')]
[('entity', 'http://www.wikidata.org/entity/Q2526255'), ('entityName', 'film director')]
[('entity', 'http://www.wikidata.org/entity/Q1742253'), ('entityName', 'movie theater operator')]
[('entity', 'http://www.wikidata.org/entity/Q373709'), ('entityName', 'boom operator')]


5

In [26]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q3406858 ?p ?obj .
OPTIONAL{?p <http://schema.org/name> ?pName .
            ?obj <http://schema.org/name> ?objName .}
} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q702269'), ('objName', 'professional')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2500638'), ('objName', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q4220920'), ('objName', 'filmmaking occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q4220920'), ('objName', 'filmmaking occupation')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('obj', 'film professional')]
[('p', 'http://www.wikidata.org/prop/direct/P2521'), ('obj', 'film professional')]


7

Node film director

In [27]:
queryString = """
SELECT ?p ?pName ?obj ?objName 
WHERE { 

{
wd:Q2526255 ?p ?obj .

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .
}

union

{

?obj ?p wd:Q2526255 .

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

}

} 
limit 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q15278146'), ('objName', 'Template:Infobox film director')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q222344'), ('objName', 'cinematographer')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1969'), ('objName', 'MovieMeter director ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3406858'), ('objName', 'film professional')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3391

30

Class director

In [28]:
queryString = """
SELECT ?p ?pName ?obj ?objName 
WHERE { 

wd:Q3455803 ?p ?obj.

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .
} 
limit 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q2137554'), ('objName', 'direction')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q2607241'), ('objName', 'mise en scène')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10809861'), ('objName', 'Template:Infobox film director')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P57'), ('objName', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2462658'), ('objName', 'manager')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q483501'), ('objName', 'artist')]
[('p', 'http://www.w

11

**3 - BGP workers for a film**
All type of workers for a film.

In [29]:
queryString = """
SELECT DISTINCT ?workerName
WHERE { 

?worker wdt:P31/wdt:279* wd:Q4220920 .

?worker <http://schema.org/name> ?workerName .

} 
ORDER BY ASC (?workerName)
limit 100
"""
#FILTER(!isLiteral(?worker))
print("Results")
run_query(queryString)

Results
[('workerName', 'Calibration')]
[('workerName', 'Camera resectioning')]
[('workerName', 'Filmare')]
[('workerName', 'Focus puller')]
[('workerName', 'Intimacy coordinator')]
[('workerName', 'Sound design')]
[('workerName', 'Sound editing')]
[('workerName', 'Special effects director')]
[('workerName', 'animation director')]
[('workerName', 'animator')]
[('workerName', 'background stylist')]
[('workerName', 'best boy')]
[('workerName', 'camera operator')]
[('workerName', 'casting director')]
[('workerName', 'character animator')]
[('workerName', 'cinematographer')]
[('workerName', 'clean-up artist')]
[('workerName', 'clean-up supervisor')]
[('workerName', 'computer animator')]
[('workerName', 'coordinating animator')]
[('workerName', 'coordinating producer')]
[('workerName', 'costume designer')]
[('workerName', 'costume technician')]
[('workerName', 'dialogue writer')]
[('workerName', 'digital imaging technician')]
[('workerName', 'documentarian')]
[('workerName', 'dolly grip')]


74

Properties of the workers just found

In [30]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?worker wdt:P31/wdt:279* wd:Q4220920 .
?worker ?p ?obj.

?worker <http://schema.org/name> ?workerName .
?p <http://schema.org/name> ?pName .

} 
ORDER BY ASC (?workerName)
limit 100
"""
#FILTER(!isLiteral(?worker))
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8519'), ('pName', 'RKD thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4652'), ('pName', 'ESCO Occupation ID')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1021'), ('pName', 'KldB-2010 occupation code')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P4839'), ('pName', 'Wolfram Language entity code')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('pName', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8814'), ('pName', 'WordNet 3.1 Synset ID')]
[('p', 'http://www.wikidata.org/prop/direct/P952'), ('pName', 'ISCO-88 occupation code')]
[('p', 'http://www.wik

76

More general approach: "Human on a film"

In [31]:
queryString = """
SELECT DISTINCT ?p ?role
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 .

?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker))
} 
limit 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1809'), ('role', 'choreographer')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('role', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('role', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('role', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('role', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P2515'), ('role', 'costume designer')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('role', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('role', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('role', 'screenwriter')]
[('p', 'http://www.wikidata.org/prop/direct/P676'), ('role', 'lyrics by')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('role', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('role', 'production designer')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'),

79

I see the properties "genere", "has effect" and "has list" as somenthing related to human... looking into it because it is strange.

In [32]:
queryString = """
SELECT DISTINCT ?role ?filmName ?workerName
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 .

?worker <http://schema.org/name> ?workerName .
?film <http://schema.org/name> ?filmName .
?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker) && (?p = wdt:P136 || ?p = wdt:P1542 || ?p = wdt:P2354))
} 
limit 100
"""

print("Results")
run_query(queryString)

Results
[('role', 'genre'), ('filmName', 'Yajamaana'), ('workerName', 'Rajesh Ramanath')]
[('role', 'genre'), ('filmName', 'Devyani'), ('workerName', 'Banish Shah')]
[('role', 'has effect'), ('filmName', 'Demasiado corazón'), ('workerName', 'Lee Wilson')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Greg Wise')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Sophia Myles')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Ed Stoppard')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Paul Kaye')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Russell Tovey')]
[('role', 'has list'), ('filmName', 'Blackwood'), ('workerName', 'Joanna Vanderham')]


9

Esploro P136... human?

In [33]:
queryString = """
SELECT ?filmName ?workerName
WHERE { 

?film wdt:P136 ?worker .
?worker wdt:P31 wd:Q5 .

?worker <http://schema.org/name> ?workerName .
?film <http://schema.org/name> ?filmName .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Henriette de Clèves (1542–1601), 4th Duchess of Nevers'), ('workerName', 'Henriette of Cleves')]
[('filmName', 'Sweet Baby James'), ('workerName', 'Peter Asher')]
[('filmName', 'Temple in rocky landscape-AA 280'), ('workerName', 'Sesshū Tōyō')]
[('filmName', 'The Close of the Silver Age – The Fruits of Jealousy'), ('workerName', 'Edward Habich')]
[('filmName', 'Jean-Baptiste Collet'), ('workerName', 'Jacques-Louis David')]
[('filmName', 'Fast as You Can'), ('workerName', 'Fiona Apple')]
[('filmName', 'The Collected Works of Theodore Parker'), ('workerName', 'Theodore Parker')]
[('filmName', 'Milla series'), ('workerName', 'Gunilla Bergström')]
[('filmName', 'Yajamaana'), ('workerName', 'Rajesh Ramanath')]
[('filmName', 'Rangiaeata. [1840]'), ('workerName', 'Te Rangihaeata')]


10

Properties of the "humans" on a set

In [34]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 .
?worker ?p1 ?obj .

?p1 <http://schema.org/name> ?p1Name .

} 

limit 70

"""
#order by asc (?p1Name)
#FILTER(!isLiteral(?p1))
print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/prop/direct/P3881'), ('p1Name', 'National Track & Field Hall of Fame athlete ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P3560'), ('p1Name', 'College Football Data Warehouse ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P1006'), ('p1Name', 'Nationale Thesaurus voor Auteurs ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P101'), ('p1Name', 'field of work')]
[('p1', 'http://www.wikidata.org/prop/direct/P102'), ('p1Name', 'member of political party')]
[('p1', 'http://www.wikidata.org/prop/direct/P103'), ('p1Name', 'native language')]
[('p1', 'http://www.wikidata.org/prop/direct/P106'), ('p1Name', 'occupation')]
[('p1', 'http://www.wikidata.org/prop/direct/P108'), ('p1Name', 'employer')]
[('p1', 'http://www.wikidata.org/prop/direct/P109'), ('p1Name', 'signature')]
[('p1', 'http://www.wikidata.org/prop/direct/P1146'), ('p1Name', 'World Athletics athlete ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P1196'), ('p1Name', 'manner of d

70

Deep dive into the workers properties. I know that the workers in the film industry "Q4220920" are instances. 

In [35]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 ; 
        wdt:P31/wdt:279* wd:Q4220920 ;
        ?p1 ?obj .

?p1 <http://schema.org/name> ?p1Name .
} 

limit 70

"""
#ORDER BY ASC (?p1Name)wdt:P106/
#FILTER(!isLiteral(?p1))
print("Results")
run_query(queryString)

Results
Empty


0

This doesn't work because the worker is a person, like Woody Allen. From one of his queries I see that the has an occupation. The occupation is contained in the film industry class. I add the P106 to the worker. <br>

**3 - Here I have the BGP of the worker in the film industry.**

In [36]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 ; 
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 ;
        ?p1 ?obj .

?p1 <http://schema.org/name> ?p1Name .
} 

limit 70

"""
#ORDER BY ASC (?p1Name)
#FILTER(!isLiteral(?p1))
print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/prop/direct/P1006'), ('p1Name', 'Nationale Thesaurus voor Auteurs ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P101'), ('p1Name', 'field of work')]
[('p1', 'http://www.wikidata.org/prop/direct/P106'), ('p1Name', 'occupation')]
[('p1', 'http://www.wikidata.org/prop/direct/P1266'), ('p1Name', 'AlloCiné person ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P1741'), ('p1Name', 'GTAA ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P18'), ('p1Name', 'image')]
[('p1', 'http://www.wikidata.org/prop/direct/P19'), ('p1Name', 'place of birth')]
[('p1', 'http://www.wikidata.org/prop/direct/P2019'), ('p1Name', 'AllMovie person ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P21'), ('p1Name', 'sex or gender')]
[('p1', 'http://www.wikidata.org/prop/direct/P213'), ('p1Name', 'ISNI')]
[('p1', 'http://www.wikidata.org/prop/direct/P214'), ('p1Name', 'VIAF ID')]
[('p1', 'http://www.wikidata.org/prop/direct/P227'), ('p1Name', 'GND ID')]
[('p1', 'http

70

In [37]:
queryString = """
SELECT DISTINCT ?p ?role
WHERE { 

?film wdt:P31 wd:Q11424 .
?film wdt:P57 wd:Q25089.
?film ?p ?worker .
?worker wdt:P31 wd:Q5 ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .

?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker))
} 
ORDER BY ASC(?role)
limit 60
"""
#
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('role', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('role', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P2515'), ('role', 'costume designer')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('role', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('role', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('role', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'), ('role', 'executive producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('role', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('role', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('role', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('role', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('role', 'production designer')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), 

13

## Task 4 - Workers in films directed by Woody Allen

**4 - Workers in films directed by Q.Tarantino**

In [38]:
queryString = """
SELECT DISTINCT ?p ?role
WHERE { 

?film wdt:P31 wd:Q11424 .
?film wdt:P57 wd:Q3772.
?film ?p ?worker .
?worker wdt:P31 wd:Q5  ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .

?worker <http://schema.org/name> ?workerName .
?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker))
} 
ORDER BY ASC(?role)
limit 60
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1877'), ('role', 'after a work by')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('role', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('role', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('role', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P2515'), ('role', 'costume designer')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('role', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('role', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'), ('role', 'executive producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('role', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('role', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('role', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('role', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'),

14

Movies of Q. Tarantino

In [39]:
queryString = """
SELECT ?film ?name 
WHERE { 

?film wdt:P31 wd:Q11424 .
?film wdt:P57 wd:Q3772.
?film <http://schema.org/name> ?name .

} 
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('name', 'The Hateful Eight')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction')]
[('film', 'http://www.wikidata.org/entity/Q1137310'), ('name', 'Death Proof')]
[('film', 'http://www.wikidata.org/entity/Q192115'), ('name', 'Sin City')]
[('film', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1')]
[('film', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds')]
[('film', 'http://www.wikidata.org/entity/Q72962'), ('name', 'Reservoir Dogs')]
[('film', 'http://www.wikidata.org/entity/Q928019'), ('name', 'Kill Bill Volume 2')]
[('film', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('name', 'Once Upon a Time in Hollywood')]
[('film', 'http://www.wikidata.org/entity/Q18126825'), ('name', 'Kill Bill: 

12

BGP Tarantino

In [40]:
queryString = """
SELECT distinct ?p ?pName 
WHERE { 

wd:Q3772 ?p ?obj .

optional{?p <http://schema.org/name> ?pName .}

} 
ORDER BY ASC(?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P245')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P269')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3348')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P349')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P434')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5034')]
[('p', 'http://www.wikidata.org/prop/direct-n

210

## Task 5

Count worker for Woody Allen

In [41]:
queryString = """
SELECT COUNT(DISTINCT (?worker) AS ?howmany)
WHERE { 

?film wdt:P31 wd:Q11424 .
?film wdt:P57 wd:Q25089.
?film ?role ?worker .
?worker wdt:P31 wd:Q5 ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .

?worker <http://schema.org/name> ?workerName .
?role <http://schema.org/name> ?roleName .

FILTER(!isLiteral(?worker))

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '739')]


1

Count worker for Q.Tarantino

In [42]:
queryString = """
SELECT COUNT(DISTINCT (?worker) AS ?howmany)
WHERE { 

?film wdt:P31 wd:Q11424 .
?film wdt:P57 wd:Q3772.
?film ?role ?worker .
?worker wdt:P31 wd:Q5 ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .

?worker <http://schema.org/name> ?workerName .
?role <http://schema.org/name> ?roleName .

FILTER(!isLiteral(?worker))

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '264')]


1

Shared actors first and basic idea. The query is inactive because it require too much time

**Shared actor between the two directors**

Adjust the previous query.

In [43]:
queryString = """
SELECT DISTINCT ?actorForWAName AS ?sharedActorName
WHERE { 
    
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P161 ?actorForWA .
    ?actorForWA <http://schema.org/name> ?actorForWAName .
    {
    SELECT ?actorForQT WHERE 
    {
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772 .
    ?film wdt:P161 ?actorForQT .
    
    }
    }
    
FILTER (?actorForQT = ?actorForWA)

} 
"""

print("Results")
run_query(queryString)

Results
[('sharedActorName', 'Mike Myers')]
[('sharedActorName', 'Tim Roth')]
[('sharedActorName', 'Steve Buscemi')]
[('sharedActorName', 'Léa Seydoux')]
[('sharedActorName', 'Rosanna Arquette')]
[('sharedActorName', 'Daryl Hannah')]
[('sharedActorName', 'Robert De Niro')]
[('sharedActorName', 'Uma Thurman')]
[('sharedActorName', 'Christopher Walken')]
[('sharedActorName', 'Leonardo DiCaprio')]
[('sharedActorName', 'Burt Reynolds')]
[('sharedActorName', 'Christoph Waltz')]


12

## Task 6

Search for budget/cost of films directed by Woody Allen

In [44]:
queryString = """
SELECT DISTINCT ?filmName ?budget
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P2130 ?budget .
    
    ?film <http://schema.org/name> ?filmName .
} 
ORDER BY DESC(?budget)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'You Will Meet a Tall Dark Stranger'), ('budget', '22000000')]
[('filmName', 'Anything Else'), ('budget', '18000000')]
[('filmName', 'Blue Jasmine'), ('budget', '18000000')]
[('filmName', 'Midnight in Paris'), ('budget', '17000000')]
[('filmName', 'To Rome with Love'), ('budget', '17000000')]
[('filmName', 'Vicky Cristina Barcelona'), ('budget', '15500000')]
[('filmName', 'Match Point'), ('budget', '15000000')]
[('filmName', 'Whatever Works'), ('budget', '15000000')]
[('filmName', 'Manhattan Murder Mystery'), ('budget', '13500000')]
[('filmName', "Cassandra's Dream"), ('budget', '13000000')]
[('filmName', 'scoop'), ('budget', '4000000')]
[('filmName', 'Annie Hall'), ('budget', '4000000')]
[('filmName', 'Bananas'), ('budget', '2000000')]


13

Search for budget/cost of films directed by Q. Tarantino

In [45]:
queryString = """
SELECT DISTINCT ?filmName ?budget
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P2130 ?budget .
    
    ?film <http://schema.org/name> ?filmName .
} 
ORDER BY DESC(?budget)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('budget', '100000000')]
[('filmName', 'Inglourious Basterds'), ('budget', '70000000')]
[('filmName', 'The Hateful Eight'), ('budget', '44000000')]
[('filmName', 'Sin City'), ('budget', '40000000')]
[('filmName', 'Kill Bill Volume 1'), ('budget', '30000000')]
[('filmName', 'Kill Bill Volume 2'), ('budget', '30000000')]
[('filmName', 'Jackie Brown'), ('budget', '12000000')]
[('filmName', 'Pulp Fiction'), ('budget', '8000000')]


8

Here I try to use the ASK query

In [46]:
queryString = """
ASK WHERE{
{
        SELECT (max (xsd:integer(?budget)) as ?ax)
            WHERE { 
                        ?film wdt:P31 wd:Q11424 .
                        ?film wdt:P57 wd:Q3772.
                        ?film wdt:P2130 ?budget .

                        ?film <http://schema.org/name> ?filmName .
                    } 
        }

FILTER (?ax > 1000)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

**6 - Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?**  <br>

Yes. "Django Unchained" had 100 millions as budget vs "You Will Meet a Tall Dark Stranger" with 22 millons.

In [47]:
queryString = """
ASK WHERE {
    
    {
        SELECT (max (xsd:integer(?budgetWA)) as ?maxBudgetWA)
            WHERE { 
                    ?film wdt:P31 wd:Q11424 .
                    ?film wdt:P57 wd:Q25089.
                    ?film wdt:P2130 ?budgetWA .

                    ?film <http://schema.org/name> ?filmName .
                } 
                
    }
    {
    
        SELECT (max (xsd:integer(?budgetQT)) as ?maxBudgetQT)
            WHERE { 
                    ?film wdt:P31 wd:Q11424 .
                    ?film wdt:P57 wd:Q3772.
                    ?film wdt:P2130 ?budgetQT .

                    ?film <http://schema.org/name> ?filmName .
                } 
                
    
    }
       
    FILTER (xsd:integer(?maxBudgetQT) > xsd:integer(?maxBudgetWA))
    
}
"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

## Task 7.1

Search for the awards. In this case I try to find the properties based on a regex. <br>
As we can see there are two interesting properties P1411 and P166. 

In [48]:
queryString = """
SELECT DISTINCT ?p2 ?p2Name
WHERE { 

wd:Q11424 ?p ?obj .
?obj ?p2 ?obj2 .
?p2 <http://schema.org/name> ?p2Name.

FILTER (REGEX(?p2Name, ".*wa.*") || REGEX(?p2Name, ".*nom.*"))
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p2', 'http://www.wikidata.org/prop/direct/P1411'), ('p2Name', 'nominated for')]
[('p2', 'http://www.wikidata.org/prop/direct/P166'), ('p2Name', 'award received')]
[('p2', 'http://www.wikidata.org/prop/direct/P6145'), ('p2Name', 'Academy Awards Database film ID')]


3

Awards of W. Allen

In [49]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P1411 ?nomination .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', "A Midsummer Night's Sex Comedy"), ('nomination', 'http://www.wikidata.org/entity/Q602866'), ('nomName', 'Golden Raspberry Award for Worst Actress')]
[('filmName', 'Alice'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103360'), ('nomName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103618'), ('nomName', 'Academy Award for Best Actress')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103916'), ('nomName', 'Academy Award for Best

64

**What prizes has W. Allen won?**

In [50]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P166 ?win .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q1011547'), ('winName', 'Golden Globe Award')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q102427'), ('winName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103360'), ('winName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName', 'Academy Award for Best Actress')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q1966965'), ('winName', 'National Board of Review: Top Ten Films')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q823422'), ('winName', 'National Film Registry')]
[('filmName', 'Blue Jasmine'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName

35

Those are all the awards. <br>
I will search only for the Academy ones(Oscars). <br> 
For my curiosity I duplicate the last two cells also for Q. Tarantino

Awards of QT

In [51]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P1411 ?nomination .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q106291'), ('nomName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q131520'), ('nomName', 'Academy Award for Best Cinematography')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q488645'), ('nomName', 'Academy Award for Best Sound Editing')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q103360'), ('nomName

63

What prizes has Q. Tarantino won?

In [52]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P166 ?win .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q1966965'), ('winName', 'National Board of Review: Top Ten Films')]
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Inglourious Basterds'), ('win', 'http://www.wikidata.org/entity/Q1966965'), ('winName', 'National Board of Review: Top Ten Films')]
[('filmName', 'Inglourious Basterds'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Jackie Brown'), ('win', 'http://www.wikidata.org/entity/Q1966965'), ('winName', 'National Board of Review: Top Ten Films')]
[('filmName', 'Kill Bill Volume 1'), ('win', 'http://www.wikidata.org/entity/Q1062026'), ('winName', 'MTV Movie Award for 

35

Search of Academy Awards: 
I take one of the awards that are listed below and I look for its class. 

In [53]:
queryString = """
SELECT DISTINCT ?obj2 ?obj2Name
WHERE { 

wd:Q11424 ?p ?obj .
?obj wdt:P1411 ?obj2 .
?obj2 <http://schema.org/name> ?obj2Name.

} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('obj2', 'http://www.wikidata.org/entity/Q1413741'), ('obj2Name', 'Saturn Award for Best Actress')]
[('obj2', 'http://www.wikidata.org/entity/Q21995139'), ('obj2Name', 'Academy Award for Best Cinematography, Black-and-White')]
[('obj2', 'http://www.wikidata.org/entity/Q1773951'), ('obj2Name', 'Empire Award for Best Actor')]
[('obj2', 'http://www.wikidata.org/entity/Q17011392'), ('obj2Name', 'Visual Effects Society Award for Outstanding Visual Effects in an Effects Driven Feature Motion Picture')]
[('obj2', 'http://www.wikidata.org/entity/Q23656761'), ('obj2Name', 'Scream Award for Best Science Fiction Actress')]
[('obj2', 'http://www.wikidata.org/entity/Q4376972'), ('obj2Name', 'Grammy Award for Best Score Soundtrack for Visual Media')]
[('obj2', 'http://www.wikidata.org/entity/Q8555'), ('obj2Name', 'Saturn Award for Best Special Effects')]
[('obj2', 'http://www.wikidata.org/entity/Q981030'), ('obj2Name', 'Saturn Award for Best Writing')]
[('obj2', 'http://www.wikidata.org/ent

59

Class of Q281939 or Q830079.

From here I find that Academy awards are represented as class Q19020

In [54]:
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {

?class wdt:P31 ?obj .

?obj <http://schema.org/name> ?objName.
FILTER (?class = wd:Q830079 || ?class = wd:Q281939)
} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('objName', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q71076284'), ('objName', 'award for best editing')]


2

**7 - BGP Academy awards**

In [55]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q19020 ?p ?obj .

?p <http://schema.org/name> ?pName .

OPTIONAL {?obj <http://schema.org/name> ?objName .}

} 
ORDER BY ASC(?pName)
"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9084'), ('pName', 'ABC News topic ID'), ('obj', 'academy-awards-oscars')]
[('p', 'http://www.wikidata.org/prop/direct/P9629'), ('pName', 'Armeniapedia ID'), ('obj', '10529')]
[('p', 'http://www.wikidata.org/prop/direct/P8295'), ('pName', 'AustLit ID'), ('obj', 'v30')]
[('p', 'http://www.wikidata.org/prop/direct/P6200'), ('pName', 'BBC News topic ID'), ('obj', 'ceq40e9pmywt')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID'), ('obj', '00000571n')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID'), ('obj', '12149710v')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('pName', 'Brockhaus Enzyklopädie online ID'), ('obj', 'oscarr')]
[('p', 'http://www.wikidata.org/prop/direct/P5905'), ('pName', 'Comic Vine ID'), ('obj', '4015-56187')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Academy Awards')]
[('p', 'ht

87

I notice that some awards are missing, like the one for Best Sound/Audio or Soundtrack... <br>

Here I select the properties of the entitis connected to Q19020.

In [56]:
queryString = """
SELECT DISTINCT ?partOf ?pName
WHERE { 

?award ?partOf wd:Q19020 .

?partOf <http://schema.org/name> ?pName .

OPTIONAL {?award <http://schema.org/name> ?awName .}

} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('partOf', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('partOf', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('partOf', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('partOf', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('partOf', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('partOf', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('partOf', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('partOf', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('partOf', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of')]
[('partOf', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('partOf', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('partOf', 'http://www.wikidata.org/prop/direct/P800

15

Finally with P31 I find all the awards. Q830079 is the awards for the best sound, one of the missing awards. 

In [57]:
queryString = """
SELECT DISTINCT ?award ?awName
WHERE { 

?award wdt:P31 wd:Q19020 .

OPTIONAL {?award <http://schema.org/name> ?awName .}

} 

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q22235305'), ('awName', 'Academy Award for Best Original Musical or Comedy Score')]
[('award', 'http://www.wikidata.org/entity/Q21995136'), ('awName', 'Academy Award for Best Cinematography, Color')]
[('award', 'http://www.wikidata.org/entity/Q21995139'), ('awName', 'Academy Award for Best Cinematography, Black-and-White')]
[('award', 'http://www.wikidata.org/entity/Q22253131'), ('awName', 'Academy Award for Best Art Direction, Color')]
[('award', 'http://www.wikidata.org/entity/Q22752811'), ('awName', 'Academy Award for Best Score, Adaptation or Treatment')]
[('award', 'http://www.wikidata.org/entity/Q8610'), ('awName', 'Academy Scientific and Technical Award')]
[('award', 'http://www.wikidata.org/entity/Q1324407'), ('awName', 'Academy Award for Best Live Action Short Film')]
[('award', 'http://www.wikidata.org/entity/Q487136'), ('awName', 'Academy Award for Best Makeup and Hairstyling')]
[('award', 'http://www.wikidata.org/entity/Q50

63

## Task 7.2

Subproperties of Academy Awards..? 
I select the properties of academy awards, then I put those properties as subject in the external select action. 

In [58]:
queryString = """
SELECT DISTINCT ?item ?itemName ?p2 ?p2Name ?p
WHERE { 
    
    ?item ?p2 ?obj2 .
    OPTIONAL{?item <http://schema.org/name> ?itemName . ?p2 <http://schema.org/name> ?p2Name}
    {
    SELECT ?p WHERE {
    wd:Q19020 ?p ?obj .
    OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
    }
    }   
    FILTER (?obj2 = ?p)
    }

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('item', 'nodeID://b275810'), ('p2', 'http://www.w3.org/2002/07/owl#onProperty'), ('p', 'http://www.wikidata.org/prop/direct/P527')]
[('item', 'http://www.wikidata.org/entity/P527'), ('p2', 'http://wikiba.se/ontology#directClaim'), ('p', 'http://www.wikidata.org/prop/direct/P527')]
[('item', 'nodeID://b272771'), ('p2', 'http://www.w3.org/2002/07/owl#onProperty'), ('p', 'http://www.wikidata.org/prop/direct/P17')]
[('item', 'http://www.wikidata.org/entity/P17'), ('p2', 'http://wikiba.se/ontology#directClaim'), ('p', 'http://www.wikidata.org/prop/direct/P17')]
[('item', 'nodeID://b221335'), ('p2', 'http://www.w3.org/2002/07/owl#onProperty'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('item', 'http://www.wikidata.org/entity/P31'), ('p2', 'http://wikiba.se/ontology#directClaim'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('item', 'nodeID://b170471'), ('p2', 'http://www.w3.org/2002/07/owl#onProperty'), ('p', 'http://www.wikidata.org/prop/direct/P279')]
[('item', 'h

143

Explore P17 to try to find the subproperties of the properties of Academy awards. 

In [59]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE { 
    
    wd:P17 ?p ?obj .
    OPTIONAL{
    ?obj <http://schema.org/name> ?objName . 
    ?p <http://schema.org/name> ?pName .
    }
    }
"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2875'), ('pName', 'property usage tracking category'), ('obj', 'http://www.wikidata.org/entity/Q20116958')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q51')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q513')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q6256')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('obj', 'http://www.wikidata.org/entity/P2012')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('obj', 'http://www.wikidata.org/entity/P131')]
[('p', 'http://www.wikidata.org/prop/direct/P1647'), ('pName', 'subproperty of'), ('obj', 'http://www.wikidata.org/entity/P131')]
[('p', 'http://www.wikidata.org/

50

P1647 is "subproperty of"

Here I use the subproperty id that I've found previously. Unfortunately this approach doesn't work. <br>

In the future I will try other approaches

In [60]:
queryString = """
SELECT DISTINCT ?p1 WHERE { 
    
    ?p1 wdt:P1647 ?obj2 .
    {
    SELECT ?p WHERE {
    wd:Q19020 ?p ?obj .
    }
    } 
    FILTER (?p = ?obj2)
    
    }

"""
#OPTIONAL{?obj2 <http://schema.org/name> ?obj2Name .?p1 <http://schema.org/name> ?p1Name}
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
Empty


0

In [61]:
queryString = """
SELECT DISTINCT ?itemName ?pName
WHERE { 
    
    ?item wdt:P1647 ?p .
    OPTIONAL{
    ?p <http://schema.org/name> ?pName.
    ?item <http://schema.org/name> ?itemName.}
    }
    
LIMIT 10

"""
#OPTIONAL{?obj <http://schema.org/name> ?objName . ?p <http://schema.org/name> ?pName}
print("Results")
run_query(queryString)

Results
[('itemName', 'software version identifier'), ('pName', 'has edition or translation')]
[('itemName', 'Fotografen.nl ID'), ('pName', 'RKDartists ID')]
[('itemName', 'exemplar of'), ('pName', 'instance of')]
[('itemName', 'heritage designation'), ('pName', 'instance of')]
[('itemName', 'legal form'), ('pName', 'instance of')]
[('itemName', 'vessel class'), ('pName', 'instance of')]
[('itemName', 'taxon rank'), ('pName', 'instance of')]
[('itemName', 'galaxy morphological type'), ('pName', 'instance of')]
[('itemName', 'list of monuments'), ('pName', 'list of works')]
[('itemName', 'discography'), ('pName', 'list of works')]


10

## Task 7

**Academy Awards di WA - Nomination**

In [62]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName 
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P1411 ?nomination .
    ?nomination wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Alice'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103360'), ('nomName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103618'), ('nomName', 'Academy Award for Best Actress')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103916'), ('nomName', 'Academy Award for Best Actor')]
[('filmName', 'Blue Jasmine'), ('nomination', 'http://www.wikidata.org/entity/Q103618'), ('nomName', 'Academy Award for Best Actress')]
[('filmName', 'Blu

52

**Academy Awards di WA - Win**

In [63]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103360'), ('winName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q102427'), ('winName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName', 'Academy Award for Best Actress')]
[('filmName', 'Blue Jasmine'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName', 'Academy Award for Best Actress')]
[('filmName', 'Bullets Over Broadway'), ('win', 'http://www.wikidata.org/entity/Q106301'), ('winName', 'Academy Award for Best Supporting Actress')]
[('filmName', 'Hannah and Her Sisters'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Hannah and 

12

Try to find the edition of a prize: Best director "Q103360"

In [64]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

    wd:Q103360 ?p ?obj .
    
    OPTIONAL{
    ?p <http://schema.org/name> ?pName .
    ?obj <http://schema.org/name> ?objName .
    }
    }

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q495980'), ('objName', 'Bong Joon-ho')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q160726'), ('objName', 'Ang Lee')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q28028'), ('objName', 'Alfonso Cuarón')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q55215'), ('objName', 'Alejandro González Iñárritu')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q6667698'), ('objName', 'Template:Academy Award for Best Director')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('objName', 'United States of America')]
[('p', 

23

Awards won by WA as a person

In [65]:
queryString = """
SELECT ?win ?winName 
WHERE { 

    wd:Q25089 wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    ?win <http://schema.org/name> ?winName .
    }
    

"""

print("Results")
run_query(queryString)

Results
[('win', 'http://www.wikidata.org/entity/Q103360'), ('winName', 'Academy Award for Best Director')]
[('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]


2

**Academy Awards di QT - Nomination**

In [66]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P1411 ?nomination .
    ?nomination wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q106291'), ('nomName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q131520'), ('nomName', 'Academy Award for Best Cinematography')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q488645'), ('nomName', 'Academy Award for Best Sound Editing')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q106291'), ('nomName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), 

24

**Academy Awards di QT - Win**

In [67]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Inglourious Basterds'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Once Upon a Time in Hollywood'), ('win', 'http://www.wikidata.org/entity/Q277751'), ('winName', 'Academy Award for Best Production Design')]
[('filmName', 'Once Upon a Time in Hollywood'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Pulp Fiction'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'The Hateful Eight'), ('win', 'http://www.wikidata.org/entity/Q488651'), ('wi

7

**Awards won by QT as a person**

In [68]:
queryString = """
SELECT ?win ?winName 
WHERE { 

    wd:Q3772 wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    ?win <http://schema.org/name> ?winName .
    }
    

"""

print("Results")
run_query(queryString)

Results
[('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]


1

I was able to fully answer to point 7...

#### **Query extra**
Count the workers of Irrational man.

In [69]:
queryString = """
SELECT COUNT(DISTINCT (?worker) AS ?howmany)
WHERE { 

wd:Q17417520 ?p ?worker .

?worker <http://schema.org/name> ?workerName .
?p <http://schema.org/name> ?pName .

FILTER (?p = wdt:P1040 || ?p = wdt:P344 || ?p = wdt:P162 ||
?p = wdt:P86 || ?p = wdt:P161 || ?p = wdt:P57 || ?p = wdt:P58 || ?p = wdt:P1431 || ?p = wdt:P2515)

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '16')]


1

In [70]:
queryString = """
SELECT DISTINCT ?worker ?workerName ?pName
WHERE { 

wd:Q17417520 ?p ?worker .

?worker <http://schema.org/name> ?workerName .
?p <http://schema.org/name> ?pName .

FILTER (?p = wdt:P1040 || ?p = wdt:P344 || ?p = wdt:P162 ||
?p = wdt:P86 || ?p = wdt:P161 || ?p = wdt:P57 || ?p = wdt:P58 || ?p = wdt:P1431 || ?p = wdt:P2515)

} 
"""

print("Results")
run_query(queryString)

Results
[('worker', 'http://www.wikidata.org/entity/Q3161523'), ('workerName', 'Jamie Blackley'), ('pName', 'cast member')]
[('worker', 'http://www.wikidata.org/entity/Q25089'), ('workerName', 'Woody Allen'), ('pName', 'director')]
[('worker', 'http://www.wikidata.org/entity/Q25089'), ('workerName', 'Woody Allen'), ('pName', 'screenwriter')]
[('worker', 'http://www.wikidata.org/entity/Q3831197'), ('workerName', 'Letty Aronson'), ('pName', 'producer')]
[('worker', 'http://www.wikidata.org/entity/Q4726932'), ('workerName', 'Alisa Lepselter'), ('pName', 'film editor')]
[('worker', 'http://www.wikidata.org/entity/Q951018'), ('workerName', 'Darius Khondji'), ('pName', 'director of photography')]
[('worker', 'http://www.wikidata.org/entity/Q435475'), ('workerName', 'Ethan Phillips'), ('pName', 'cast member')]
[('worker', 'http://www.wikidata.org/entity/Q185140'), ('workerName', 'Joaquin Phoenix'), ('pName', 'cast member')]
[('worker', 'http://www.wikidata.org/entity/Q204586'), ('workerName',

17

In [71]:
queryString = """
SELECT DISTINCT ?workerName ?role
WHERE { 

wd:Q17417520 ?p ?worker .
?worker wdt:P31 wd:Q5 .

?worker <http://schema.org/name> ?workerName .
?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker))
} 
limit 100
"""

print("Results")
run_query(queryString)

Results
[('workerName', 'Jamie Blackley'), ('role', 'cast member')]
[('workerName', 'Woody Allen'), ('role', 'director')]
[('workerName', 'Woody Allen'), ('role', 'screenwriter')]
[('workerName', 'Letty Aronson'), ('role', 'producer')]
[('workerName', 'Alisa Lepselter'), ('role', 'film editor')]
[('workerName', 'Darius Khondji'), ('role', 'director of photography')]
[('workerName', 'Ethan Phillips'), ('role', 'cast member')]
[('workerName', 'Joaquin Phoenix'), ('role', 'cast member')]
[('workerName', 'Santo Loquasto'), ('role', 'production designer')]
[('workerName', 'Parker Posey'), ('role', 'cast member')]
[('workerName', 'Meredith Hagner'), ('role', 'cast member')]
[('workerName', 'Nancy Giles'), ('role', 'cast member')]
[('workerName', 'David Aaron Baker'), ('role', 'cast member')]
[('workerName', 'Henry Stram'), ('role', 'cast member')]
[('workerName', 'Ben Rosenfield'), ('role', 'cast member')]
[('workerName', 'David Pittu'), ('role', 'cast member')]
[('workerName', 'Robert Petko

17

In [72]:
queryString = """
SELECT DISTINCT ?filmName ?role 
WHERE { 

wd:Q25089 wdt:P106 ?obj .
?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .

FILTER (?p = wdt:P800 || ?p = wdt:P106)

} 
"""

print("Results")
run_query(queryString)

Results
[]


1